15.077/IDS.147 Problem Set 5 <br>
**Name:** Chun-Hei Lam. **ID:** 928931321 <br>
**Declaration:** I pledge that the work submitted for this coursework is my own unassisted work unless stated otherwise. <br>
**Acknowledgement to:** Harry Yu

In [2]:
import numpy as np
import gzip
import pandas as pd
import matplotlib.pyplot as plt

In [49]:
import sklearn.linear_model
import sklearn.neighbors
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [57]:
seed = 1234
misclass_scorer = make_scorer(accuracy_score)

def sign_loss(y_true, y_pred):
    diff = 1-np.sum(np.abs(np.heaviside(y_true,0) - np.heaviside(y_pred,0)))/len(y_true)
    return diff

misclass_sign_scorer = make_scorer(sign_loss)

# HTF 2.8 Handwriting Data
Compare the classification performance of linear regression and k–nearest neighbor classification on the `zipcode` data. **In particular, consider only the $2$’s and $3$’s, and $k = 1, 3, 5, 7$ and $15$.** Show both the training and test error for each choice.

*Solution: First preprocess the data.*

In [50]:
train = pd.read_csv("zip.train.gz", delim_whitespace=True, header=None)
train = train[(train[0] == 2) | (train[0] == 3)].sort_values(by=0)
X_train = train.iloc[:, 1:]
y_train = train[0].copy()
y_train[y_train == 2] = -1
y_train[y_train == 3] = 1

test = pd.read_csv("zip.test.gz", delim_whitespace=True, header=None)
test = test[(test[0] == 2) | (test[0] == 3)].sort_values(by=0)
X_test = test.iloc[:, 1:]
y_test = test[0].copy()
y_test[y_test == 2] = -1
y_test[y_test == 3] = 1

*Performance of linear classifier.*

In [66]:
linear_clf = sklearn.linear_model.RidgeClassifier(alpha=0).fit(X_train, y_train) #ridge with alpha = 0 is OLS
train_score = linear_clf.score(X_train, y_train)
test_score = linear_clf.score(X_test, y_test)
print(f"train score = {train_score}, test score = {test_score}")

train score = 0.994240460763139, test score = 0.9587912087912088


*Performance of kNN classifier*

In [69]:
for k in (1,3,5,7,15):
    kNN_clf = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k).fit(X_train, y_train)
    train_score = kNN_clf.score(X_train, y_train)
    test_score = kNN_clf.score(X_test, y_test)
    print(f"k = {k}, train score = {train_score}, test score = {test_score}")

k = 1, train score = 1.0, test score = 0.9752747252747253
k = 3, train score = 0.9949604031677466, test score = 0.9697802197802198
k = 5, train score = 0.994240460763139, test score = 0.9697802197802198
k = 7, train score = 0.9935205183585313, test score = 0.967032967032967
k = 15, train score = 0.9906407487401008, test score = 0.9615384615384616


# HTF 7.5 Linear Smoother
For a linear smoother $\hat{\vec{y}} = S\vec{y}$, show that
\begin{equation}
\sum_{i=1}^N \text{Cov}(\hat{y}_i, y_i) = \text{tr}(S) \sigma^2_\epsilon
\end{equation}
which justifies its use as the effective number of parameters.

*Solution: Just note that*
\begin{equation}
\sum_{i=1}^N \text{Cov}(\hat{y}_i, y_i) = \sum_{i=1}^N [\text{Cov}(S\vec{y}, \vec{y})]_{ii} = \sum_{i=1}^N [S\text{Cov}(\vec{y}, \vec{y})]_{ii} = \sum_{i=1}^N [S \sigma^2_\epsilon I]_{ii} = \text{tr}(S) \sigma^2_\epsilon
\end{equation}

# HTF 3.17 Spam Data
Estimated coefficients and test error results, for different subset and shrinkage methods applied to the `spam.txt` data. The blank entries correspond to variables omitted. **Best subset regression is NOT required.**

*Solution: Preprocessing*

In [4]:
df = pd.read_csv("spam.txt", delim_whitespace=True, header=None)
X = df.iloc[:,:57].copy()
y = df.iloc[:,57].copy()
y[y==0] = -1

*For each classifier return the coefficients and test errors for 10 folds CV.*

In [58]:
# OLS
np.random.seed(seed)
ols_clf = sklearn.linear_model.LinearRegression()
ols_scores = cross_val_score(ols_clf, X, y, cv=10, scoring=misclass_sign_scorer)
print(f"test score = {ols_scores.mean()} +/- {ols_scores.std()}")

test score = 0.8500792228614543 +/- 0.11245239140272323


In [67]:
# Ridge
np.random.seed(seed)
ridge_clf = sklearn.linear_model.RidgeClassifierCV(alphas=np.logspace(-3,3,num=7))
ridge_scores = cross_val_score(ridge_clf, X, y, cv=10, scoring=misclass_scorer)
print(f"test score = {ridge_scores.mean()} +/- {ridge_scores.std()}")

test score = 0.8826360464019617 +/- 0.01871343616407231


array([0.87418655, 0.8826087 , 0.85869565, 0.90217391, 0.88043478,
       0.89782609, 0.91956522, 0.88695652, 0.86086957, 0.86304348])

In [69]:
ridge_clf.fit(X,y).__dict__

{'alphas': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
 'fit_intercept': True,
 'normalize': False,
 'scoring': None,
 'cv': None,
 'gcv_mode': None,
 'store_cv_values': False,
 'alpha_per_target': False,
 'class_weight': None,
 'n_features_in_': 57,
 '_label_binarizer': LabelBinarizer(neg_label=-1),
 'alpha_': 10.0,
 'best_score_': -0.4480910418567984,
 'coef_': array([[-9.49604285e-02, -2.39306061e-02,  7.77636486e-02,
          2.39186620e-02,  1.68352912e-01,  2.33021201e-01,
          4.21380837e-01,  1.86650644e-01,  1.41900693e-01,
          3.04785798e-02,  1.09452907e-01, -5.55903562e-02,
          2.65804799e-02,  8.97933359e-03,  4.11109140e-02,
          1.50486260e-01,  1.04641832e-01,  1.10113486e-01,
          2.86645551e-02,  1.23364833e-01,  1.06064623e-01,
          8.82548004e-02,  3.46682305e-01,  1.80143926e-01,
         -4.66129486e-02, -4.40852596e-02, -2.44000986e-02,
          6.89156145e-03, -1.42308353e-02, -1.00385830e-01,
         -4.35

In [60]:
# LASSO
np.random.seed(seed)
lasso_clf = sklearn.linear_model.LassoCV(alphas=np.logspace(-3,3,num=7))
lasso_scores = cross_val_score(lasso_clf, X, y, cv=10, scoring=misclass_sign_scorer)
print(f"test score = {lasso_scores.mean()} +/- {lasso_scores.std()}")

test score = 0.8476898047722343 +/- 0.1151004063745352


In [ ]:
# PCR

In [ ]:
# PLS

# HTF 3.30 Elastic Net and Lasso
Consider the elastic-net optimization problem:

\begin{equation}
\min_\beta \left( \|\vec{y} - X\vec{\beta} \|^2 + \lambda \left( \alpha \| \beta \|_2^2 + (1-\alpha) \|\beta\|_1 \right)\right)
\end{equation}

Show how one can turn this into a lasso problem, using an augmented version of $X$ and $\vec{y}$.

*Solution: Let $\tilde{X} = \begin{pmatrix} X \\ \sqrt{\lambda \alpha} I \end{pmatrix}$ and $\tilde{Y} = \begin{pmatrix} \vec{y} \\ \vec{0} \end{pmatrix}$. Then we have 
\begin{equation}
\|\tilde{y} - \tilde{X}\vec{\beta} \|^2_2 + \lambda (1-\alpha) \|\beta\|_1 = \| \vec{y} - X\vec{\beta} \|^2_2 + (\sqrt{\alpha \lambda})^2 \|\vec{\beta} \|^2_2 + \lambda (1-\alpha) \|\beta\|_1
\end{equation}*
We have thus reduce an elastic net problem to a LASSO problem.